# Import

In [2]:
import csv
import os
import sys

import numpy as np
import pandas as pd
from bertopic import BERTopic
from tqdm import tqdm

from embedded_topic_model.utils import preprocessing, embedding
from embedded_topic_model.models.etm import ETM


ModuleNotFoundError: No module named 'embedded_topic_model'

## Add configuration file

In [ ]:
sys.path.append("/home/jovyan/core/config/")
sys.path.append("/home/jovyan/core/util/")

In [ ]:
from ALL import config
import MultilayerDict as md
from util import *

## Set condition

In [ ]:
tqdm.pandas()
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 50)

In [ ]:
s3 = S3Manager()

In [ ]:
sampling_nums = [1, 2, 4, 8, 16, 32, 64, 128]

In [3]:
# data_types = [f"20NewsSampled{sampling_num}" for sampling_num in sampling_nums]
data_types = ["TweetTopic", "TweetFinance"]
clustering_model = "ETM"

In [45]:
data_type = data_types[0]

In [46]:
model_nums = config["clustering"][clustering_model]["max_model_num"]
topic_nums = {
    data_type: config["data"][data_type_classifier(data_type)]["class_num"]
    for data_type in data_types
}

# Read data

In [47]:
df_path = s3.download(f"Preprocessing/{data_type}/master.csv")

In [48]:
df = pd.read_csv(df_path[0], index_col=0)

In [49]:
labels_path = s3.download(f"Preprocessing/{data_type}/class.csv")

In [50]:
label = df["class"].to_numpy()

In [51]:
with open(labels_path[0], mode="r") as f:
    reader = csv.reader(f)
    class_labels = [label for label in reader]

In [52]:
base_pred_path = f"Clustering/{data_type}/BERTopic/pred"
base_model_path = f"Clustering/{data_type}BERTopic/model"
base_prob_path = f"Postprocessing/{data_type}/BERTopic/prob"